# Core Statistics Using Python
### Hana Choi, Simon Business School, University of Rochester


# Assignment 3 Solutions

## Required packages

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
from scipy import optimize

# Question 1

- These solutions replicate the results from the Excel solutions ("PS3SolnQ1.xlsx")

## Loading data

- The data are constructed by hand
- See bottom of this file for a version that scrapes data from the web

In [ ]:
capm_data = pd.read_csv("/Users/hanachoi/Dropbox/teaching/core_statistics/Assignments/Solutions/PS3SolnQ1_data.csv")

# Display the first few rows of the dataframe
print(capm_data.head())

## Running regression analysis for each company

In [ ]:
# HD
hd_fit = smf.ols('R_HD ~ Mkt_Rf', data = capm_data).fit()
print(hd_fit.summary())

In [ ]:
# AAPL
aapl_fit = smf.ols('R_AAPL ~ Mkt_Rf', data = capm_data).fit()
print(aapl_fit.summary())

In [ ]:
# VZ
vz_fit = smf.ols('R_VZ ~ Mkt_Rf', data = capm_data).fit()
print(vz_fit.summary())

In [ ]:
# CSCO
csco_fit = smf.ols('R_CSCO ~ Mkt_Rf', data = capm_data).fit()
print(csco_fit.summary())

## More elegant way of writing the above code

- When writing code, we want to avoid repeatedly writing the same code.
- In this assignment, we are conducting regression analyses repeatedly for different companies.
- To minimize redundancy and enhance code readability, we can define a function that runs linear regression and prints out the summary.

In [ ]:
# Define function to run regression and print summary
def run_regression(data, y, x):
    model = smf.ols(f'{y} ~ {x}', data=data).fit()
    print(model.summary())
    return model

In [ ]:
# Repeat regression analysis for multiple companies
HD = run_regression(capm_data, 'R_HD', 'Mkt_Rf')
AAPL = run_regression(capm_data, 'R_AAPL', 'Mkt_Rf')
VZ = run_regression(capm_data, 'R_VZ', 'Mkt_Rf')
CSCO = run_regression(capm_data, 'R_CSCO', 'Mkt_Rf')

# Question 2

## Loading data

In [ ]:
rfj_data = pd.read_csv("/Users/hanachoi/Dropbox/teaching/core_statistics/Data/rfj_data.csv")

# Display the first few rows of the dataframe
print(rfj_data.head())

## Question 2 (a):  Run simple linear regressions and compute price elasticities

### Tropicana

In [ ]:
trop_fit = smf.ols('q1 ~ p1', data = rfj_data).fit()
print(trop_fit.summary())

trop_elasticity = trop_fit.params.iloc[1] * rfj_data['p1'].mean() /rfj_data['q1'].mean()
print("Elasticity Tropicana:", trop_elasticity)

### Minute Maid

In [ ]:
mm_fit = smf.ols('q2 ~ p2', data = rfj_data).fit()
print(mm_fit.summary())

mm_elasticity = mm_fit.params.iloc[1] * rfj_data['p2'].mean() /rfj_data['q2'].mean()
print("Elasticity Minute Maid:", mm_elasticity)

### Private Label

In [ ]:
priv_fit = smf.ols('q3 ~ p3', data = rfj_data).fit()
print(priv_fit.summary())

priv_elasticity = priv_fit.params.iloc[1] * rfj_data['p3'].mean() /rfj_data['q3'].mean()
print("Elasticity Private Label:", priv_elasticity)

### More elegant way of writing the above code

In [ ]:
# Define function to calculate price elasticity
def calculate_elasticity(model, data, p_var, q_var):
    slope = model.params.iloc[1]
    intercept = model.params.iloc[0]
    elasticity = slope * data[p_var].mean() / data[q_var].mean()
    return slope, intercept, elasticity

# Run regression and print summary
# We already defined run_regression function in Q1 so we can re-use it
trop_fit = run_regression(rfj_data, 'q1', 'p1') 
mm_fit = run_regression(rfj_data, 'q2', 'p2')
priv_fit = run_regression(rfj_data, 'q3', 'p3')

# Calculate price elasticities and print the results
trop_slope, trop_int, trop_elasticity = calculate_elasticity(trop_fit, rfj_data, 'p1', 'q1')
mm_slope, mm_int, mm_elasticity = calculate_elasticity(mm_fit, rfj_data, 'p2', 'q2')
priv_slope, priv_int, priv_elasticity = calculate_elasticity(priv_fit, rfj_data, 'p3', 'q3')

print("Elasticity Tropicana:", trop_elasticity)
print("Elasticity Minute Maid:", mm_elasticity)
print("Elasticity Private Label:", priv_elasticity)

## Question 2 (b):  Find the optimal price for each brand

In [ ]:
# Define the profit function
def profit(p, beta0, beta1):
    profit = (p - 0.01) * (beta0 + beta1 * p)
    return profit

In [ ]:
# Function to maximize profit by minimizing the negative profit
def maximize_profit(beta0, beta1):
    # Minimize the negative profit
    result = optimize.minimize_scalar(lambda p: -profit(p, beta0, beta1), bounds=(0, 1000), method='bounded')
    return result

In [ ]:
# Maximize profits to find optimal price for each brand
# We have previously stored the intercept and slope variables from the output of the calculate_elasticity() function in section 4.2.4.
optimal_price_trop = maximize_profit(trop_int, trop_slope)
optimal_price_mm = maximize_profit(mm_int, mm_slope)
optimal_price_priv = maximize_profit(priv_int, priv_slope)

# Print optimal prices
print("Optimal Price for Tropicana:", optimal_price_trop.x)
print("Optimal Price for Minute Maid:", optimal_price_mm.x)
print("Optimal Price for Private Label:", optimal_price_priv.x)

# Alternative Solutions for Q1

- Here I will present alternative solutions that scrape the data from the web, use different choices for the market index and risk free rate, and allow you to change the time frequency used.

## Scrapping data using yfinance

- yfinance is a package that offers a method of downloading historical market data from Yahoo! Finance's API.
- To utilize yfinance, students must first install the package by executing `conda install yfinance` in terminal within our virtual environment, which is the recommended practice for package installation. 
- Once installed, we can import this package and use the the functionalities offered by it.

### Import library

In [ ]:
# Importing yfinance
import yfinance as yf

### Fetch data

In [ ]:
# Define the ticker symbols
# I am using the SP500 for the Market Return and IRX for the risk free rate
# "^IRX": 13 week treasury bill
# "^GSPC": S&P 500
symbols = ["^IRX", "^GSPC", "HD", "AAPL", "VZ", "CSCO"]

# Fetch historical data from Yahoo Finance
# Note that you can change time interval below to something other than monthly.
data = {symbol: yf.download(symbol, start='1990-01-01', end='1999-12-31', interval='1mo') for symbol in symbols}

# Now 'data' dictionary contains the historical data for each symbol

## Prepare the dataset

In [ ]:
# Calculate monthly returns and convert IRX to the appropriate frequency.
def calculate_returns(df):
    return df['Adj Close'].pct_change()

# Prepare returns DataFrame
returns = pd.DataFrame({symbol: calculate_returns(data[symbol]) for symbol in symbols})

# Note that the "Treasury Yield" Yahoo shows is the annual "return rates" (not prices)
# As the data collected is monthly, we will need to convert yearly IRX to monthly.
# If you choose a different time interval than monthly, you need to change the "timeunit" value below accordingly.
# For weekly, set to 52 (daily leads to some problems with the IRX index)
# Here I am setting it equal to 12 for monthly data.
timeunit = 12
returns['RF'] = (1 + data['^IRX']['Adj Close'] / 100) ** (1 / timeunit) - 1 # This formula accounts for compound.

## Regression Analyses

### CAPM with no risk free rate included

In [ ]:
# First let's run CAPM with no risk free rate included
# We can again re-use run_regression function defined in Q1.
for symbol in ["HD", "AAPL", "VZ", "CSCO"]:
    # patsy Q("") is a way to ‘quote’ variable names, especially ones that do not otherwise meet Python’s variable name rules.
    run_regression(returns, symbol, 'Q("^GSPC")')    

### CAPM with constant Rf rate

In [ ]:
# Now let's use a constant RF rate
# I am setting it equal to the sample average of RF
rfrate = returns['RF'].mean()
returns['^GSPC_RF'] = returns['^GSPC'] - rfrate
for symbol in ["HD", "AAPL", "VZ", "CSCO"]:
    returns[f"{symbol}_RF"] = returns[symbol] - rfrate
    run_regression(returns, f"{symbol}_RF", 'Q("^GSPC_RF")')
    
# Using a fixed Rf rate does not change the calculation of beta at all.
# It simply shifts the intercept - do you understand why?

### CAPM with time varying Rf rate

In [ ]:
# Finally, let's allow for a time varying risk free rate
returns['^GSPC_RFt'] = returns['^GSPC'] - returns['RF']
for symbol in ["HD", "AAPL", "VZ", "CSCO"]:
    returns[f"{symbol}_RFt"] = returns[symbol] - returns['RF']
    run_regression(returns, f"{symbol}_RFt", 'Q("^GSPC_RFt")')